In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import urllib.request
import json
import requests
import datetime
import math
import gensim
import regex
import time
import re
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
df = pd.read_csv('크롤링 - 카테고리.csv', encoding='utf-8-sig')
del df['Unnamed: 11']
df.columns = ['술 종류','술 이름','도수','용량(ml)','가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가','별점 - 점수별 개수 평균(점수*개수/전체개수)','탄산','맛','향','리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기)','선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)']
df.drop([0], axis=0, inplace=True)
df.dropna(axis=0, inplace = True)
df

,술 종류,술 이름,도수,용량(ml),"가격 - 공식홈페이지 가격(1병 기준), 없으면 네이버 스토어 최저가",별점 - 점수별 개수 평균(점수*개수/전체개수),탄산,맛,향,리뷰 빈도수(리뷰개수 가장 적은걸로 개수 통일화 후 랜덤으로 뽑기),"선물용(디자인, 병모양, 이쁨, 패키지, 설날, 크리스마스 등)"
1,막걸리,배상면주가 느린마을 막걸리,6,1000,10000,4.8,O,"고소하다\n뽀얗다\n담백하다\n크림,우유(부드럽다)\n달맛(달다,달짝지근하다,달달하...",사과\n요구르트\n곡물\n과일,18923,"선물','추석','설날','크리스마스','집들이','기념','대접'"
2,막걸리,복순도가,6.5,935,12000,4.8,O,산뜻하다\n스파클링(탄산)\n꿀떡꿀떡\n순하다\n달짝지근하다(달달)\n부드럽다\n고...,청포도\r\n누룩\r\n포도\r\n복숭아\r\n향기,27821,"부모님','가족','지인','친구','명절','추석'"
4,막걸리,죽향도가,6,600,3800,4.8,X,부드럽다\r\n순하다\r\n굉장하다\r\n깔끔하다\r\n달콤하다\r\n달달\r\n중...,꿀향,5916,생일\r\n부모님\r\n가족\r\n친구\r\n명절\r\n추석
5,과실주,배상면주가 빙탄복,7,370,4000,4.8,o,탄산음료같다\n도수가높지않다\n뒷맛이 깔끔,복분자,7754,혼술
6,과실주,산내들 소백산 스위트 와인,12,750,15500,4.76,x,부드러움\n목넘김\n,향긋하다,3541,"초보자용, 가격대비, 고급스러움(맛, 포장)"
7,과실주,금이산농원 복숭아 와인,12,500,20000,4..82,x,달달하다\n신맛\n깔끔하다\n자연스러움,복숭아향 약함\n은은하게 퍼짐,3794,홈파티용
10,청주,고메기 맑은술,16,500,30000,4.83,X,깔끔하다\r\n달콤하다\r\n은은하다\r\n부드럽다\r\n달달\r\n산미\r\n맑다...,과일\n화이트 와인\n과실\n,1468,스타일\n고급스럽다\n꼼꼼하다\n추석\r\n패키지\r\n이쁘다 \r\n예쁘다\r\n...
11,청주,고헌정 동학 1957 특선 13도,13,375,3500,4.84,X,단맛\r\n가볍다\r\n은은하다\r\n부드럽다\r\n달달\r\n사케\r\n드라이\r,누룩향,1349,고급스럽다\n디자인\n예쁘다\r\n
12,청주,양촌양조 무농약 우렁이쌀,14,500,16000,4.86,X,가볍다\n반주\n달달\n단맛\n부드럽다\n순하다\n,약초향\r\n고구마향\r\n약재\r,805,선물\r\n박스\r\n술잔\r\n사은\r
13,청주,우포의아침,14,700,4500,4.7,X,부드러움\n청주\n드라이\n달달\n도수\n,누룩향 미미,482,선물\r\n요리\r\n친구\r


In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from eunjeon import Mecab